In [2]:
import numpy as np
import pandas as pd
import math
import synthetic_generate as sg
np.set_printoptions(precision=2)


def main():
    data_type="GAUSSIAN" 
    d_est=10
    d_test=64
    est_type="sw"
    repeat=50
    repeat_est=repeat
    N=10000
    low,high = -3, 3
    # set index of bins
    beta = 1

    # generate data
    data = sg.generate_synthetic_data(data_type=data_type,
            low=low,high=high,n=N,beta=beta)
    
    histo_true_est,_ = np.histogram(a=data, range=(0,beta), bins=d_est)
    q_true_est = np.reshape(histo_true_est/np.sum(histo_true_est),(1,d_est))
    
    print("synthetic data generated, N=%d." % (data.size))

    # set epsilon
    eps_grid = np.linspace(0.5,2.5,5)

    result = np.zeros((eps_grid.size, 4))
    for i in range(len(eps_grid)):

        print('\n==== eps=%.2f, repeat=%d ====' %(eps_grid[i],repeat))
        
        # _,_,q_est = sg.DP_dist_estimation(histo_true=histo_true,bin_idxs=bin_idxs,
        #     est_type=est_type,eps=eps_grid[i]*ratio, repeat=repeat_est,q_est=q_true)
        # print('initial estimation complete, eps=%.2f' 
        #       %(eps_grid[i]*ratio))

        result[i,0],result[i,1],_ = sg.DP_dist_estimation_aaa(data=data, bin_max=beta, d_est=d_est, d_test=d_test, 
                                                              eps=eps_grid[i], repeat=repeat_est, q_est=q_true_est)
        print('aaa estimation complete')
        
        result[i,2],result[i,3],_ = sg.DP_dist_estimation(data=data, bin_max=beta, d_est=d_test,est_type='sw',
                                                          eps=eps_grid[i], repeat=repeat_est)
        print('%s estimation complete')
    
    filename = 'data/result_%s_%s_%d_%d(%d).csv' % (data_type, est_type, d_est,d_test, repeat)
    temp = {'eps_grid': eps_grid,
        'mse_aaa': result[:, 0],
        'wass_aaa': result[:, 1],
        'mse_est': result[:, 2],
        'wass_est': result[:, 3]}
    pd.DataFrame(temp).to_csv(filename)
    print('task complete!')
    return temp

if __name__=='__main__':
    result = main()


synthetic data generated, N=10000.

==== eps=0.50, repeat=50 ====


ValueError: operands could not be broadcast together with shapes (1,64) (10,) (1,64) 